In [199]:
import os
import requests
import urllib.parse
import datetime
import pandas as pd

In [136]:
filtered_data = []

for match in response_matchids.json():
    url_match = f'https://europe.api.riotgames.com/lol/match/v5/matches/{match}?api_key={api_key}'
    response_match = requests.get(url_match)
    match_data = response_match.json()
    my_summoner = match_data['metadata']['participants'].index(puuid)
    team_id_my_summoner = match_data['info']['participants'][my_summoner]['teamId']
    for participant in match_data['info']['participants']:
        if (participant['teamPosition'] == 'BOTTOM' or participant['teamPosition'] == 'UTILITY') and participant['teamId'] == team_id_my_summoner and participant['puuid'] != puuid_lukas:
            filtered_data.append(participant)

df_filtered = pd.DataFrame(filtered_data)


In [137]:
df_filtered.loc[df_filtered['deaths'] == 0, 'deaths'] += 1

df_filtered['kda'] = (df_filtered['kills'] + df_filtered['assists']) / df_filtered['deaths']
df_filtered['kda'].mean()

3.0033833870955084

In [176]:
def datetime_converter(type, time):
    if type == 'date':
        timestamp = int(time) / 1000
        date = datetime.datetime.fromtimestamp(timestamp)
        return date
    elif type == 'timestamp':
        date = datetime.datetime.strptime(time, '%Y-%m-%d')
        timestamp = int(date.timestamp())
        return timestamp
    else:
        raise ValueError("Invalid type. Supported types are 'date' and 'timestamp'.")


In [143]:
def get_puuid(game_name, name_tag):
    with open('../api_key.txt', 'r') as file:
        api_key = file.read().replace('\n', '')
    url_puuid = f'https://europe.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name}/{name_tag}?api_key={api_key}'
    response_puuid = requests.get(url_puuid)
    return response_puuid.json()['puuid']

In [195]:
def get_matchids(puuid, amount_of_games=10000, starttime=0, endtime=0, type='ranked'):
    with open('../api_key.txt', 'r') as file:
        api_key = file.read().replace('\n', '')
    
    match_ids = []
    count = 0
    start = 0
    
    while count < amount_of_games:
        remaining_games = amount_of_games - count
        games_to_fetch = min(remaining_games, 100)
        
        if starttime == 0 and endtime == 0:
            url_matchids = f'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type={type}&start={start}&count={games_to_fetch}&api_key={api_key}'
        elif starttime != 0 and endtime == 0:
            url_matchids = f'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?startTime={starttime}&type={type}&start={start}&count={games_to_fetch}&api_key={api_key}'
        else:
            url_matchids = f'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?startTime={starttime}&endTime={endtime}&type={type}&start={start}&count={games_to_fetch}&api_key={api_key}'
        
        response_matchids = requests.get(url_matchids)
        match_ids.extend(response_matchids.json())
        
        count += games_to_fetch
        start += games_to_fetch
    
    return match_ids


In [168]:
list = get_matchids(puuid)
len(list)

405

In [157]:
get_matchids(puuid, 50, starttime = datetime_converter('timestamp', '2024-01-30'))

['EUW1_6924406987',
 'EUW1_6924361135',
 'EUW1_6924328766',
 'EUW1_6921801705',
 'EUW1_6921772362',
 'EUW1_6921713287',
 'EUW1_6921640887',
 'EUW1_6921441131',
 'EUW1_6921348343',
 'EUW1_6920891682',
 'EUW1_6918548140',
 'EUW1_6918502374',
 'EUW1_6916346547',
 'EUW1_6916291954',
 'EUW1_6916201256',
 'EUW1_6914047346',
 'EUW1_6914021906',
 'EUW1_6913991531',
 'EUW1_6913582627',
 'EUW1_6913517111',
 'EUW1_6913398250',
 'EUW1_6913310666',
 'EUW1_6912085408',
 'EUW1_6912005040',
 'EUW1_6911905479',
 'EUW1_6911841050',
 'EUW1_6911724507',
 'EUW1_6910562517',
 'EUW1_6910496037',
 'EUW1_6910226012',
 'EUW1_6910148040',
 'EUW1_6908742927',
 'EUW1_6908679172',
 'EUW1_6908607786',
 'EUW1_6908530191',
 'EUW1_6908475206',
 'EUW1_6908373217',
 'EUW1_6908186698',
 'EUW1_6907941399',
 'EUW1_6907379505',
 'EUW1_6907277278',
 'EUW1_6907195860',
 'EUW1_6905633629',
 'EUW1_6905533103',
 'EUW1_6905447158',
 'EUW1_6905389317',
 'EUW1_6905341069',
 'EUW1_6905294929',
 'EUW1_6904050481',
 'EUW1_6903294776']

In [169]:
list

['EUW1_6924406987',
 'EUW1_6924361135',
 'EUW1_6924328766',
 'EUW1_6921801705',
 'EUW1_6921772362',
 'EUW1_6921713287',
 'EUW1_6921640887',
 'EUW1_6921441131',
 'EUW1_6921348343',
 'EUW1_6920891682',
 'EUW1_6918548140',
 'EUW1_6918502374',
 'EUW1_6916346547',
 'EUW1_6916291954',
 'EUW1_6916201256',
 'EUW1_6914047346',
 'EUW1_6914021906',
 'EUW1_6913991531',
 'EUW1_6913582627',
 'EUW1_6913517111',
 'EUW1_6913398250',
 'EUW1_6913310666',
 'EUW1_6912085408',
 'EUW1_6912005040',
 'EUW1_6911905479',
 'EUW1_6911841050',
 'EUW1_6911724507',
 'EUW1_6910562517',
 'EUW1_6910496037',
 'EUW1_6910226012',
 'EUW1_6910148040',
 'EUW1_6908742927',
 'EUW1_6908679172',
 'EUW1_6908607786',
 'EUW1_6908530191',
 'EUW1_6908475206',
 'EUW1_6908373217',
 'EUW1_6908186698',
 'EUW1_6907941399',
 'EUW1_6907379505',
 'EUW1_6907277278',
 'EUW1_6907195860',
 'EUW1_6905633629',
 'EUW1_6905533103',
 'EUW1_6905447158',
 'EUW1_6905389317',
 'EUW1_6905341069',
 'EUW1_6905294929',
 'EUW1_6904050481',
 'EUW1_6903294776',


In [196]:
def get_match_data(match_id):
    try:
        with open('../api_key.txt', 'r') as file:
            api_key = file.read().replace('\n', '')
        url_match = f'https://europe.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
        response_match = requests.get(url_match)
        response_json = response_match.json()
        
        if 'status' in response_json and response_json['status']['status_code'] == 404:
            print('Data not found - match file not found')
            return None
        
        return response_json
    except Exception as e:
        print(f'Error occurred: {str(e)}')
        return None


In [200]:
get_match_data(list[1])

{'metadata': {'dataVersion': '2',
  'matchId': 'EUW1_6924361135',
  'participants': ['UGH-WO8yyREpnkHiXGZCSCdO75hZWzHQKM76uhIb8rInqJHH-UaLiRRru7zKrNff6lvT9gTOILgu3w',
   'SyA7_17Xydzj8wVRjsw_b0EO5S5vKL7KQmHWjwUwjp5M3i0ni6eXp2kCv_Yy9LHiYIBH_BIehH_jPw',
   'Kj-1u-IuPBlhpZX20rBervlCD54rbvoYgvQaSFhPpiLBUdIRSMNlLegRqapBOXghnLjqLe_4FSAQmw',
   'R9IQ2iSZ1Zqoeb9Xp7luKG7UYbjB0NOeCHCNHQk2XnyxQEBmUqou9WltJwSpdxJeUqXgr_fQUVmpmQ',
   '9Ht0CWnqxnim47PjTCkyMBpzhtWEAxUG9cKhJL2CTfggSjlMx6JOBgX9JPrINXZ82byBA3zpFnhmmQ',
   'lSm_LgpsT_rk6r_WtYkrVtvNx3XrgzxZlvshpnb6a_JT2OGG7JLjV-T1le46XS5zVwddL4b3GjgfoQ',
   'ZoODA_mjb6p1andETM52-Ts53SYVA3zm7dwhndrl6OLLvbb_f3-YtVYEBImDSkQmt3N4MNI2yhXB1Q',
   'wsuW4YH7WGgbZgCNwA0UZuFHdXsmBoWuTlC2NREjZIPx0Oj28mlxdp9ffzOxCXXCYZtTOFsJ8Bxb5g',
   'Lyr9Rm16FLfZatQR_eF0SiELFa4aE-vTjS2K1o83hxpC9_RB96VFwsAxM2HGy57dEiB4eGDVh9aZhw',
   'vqQT7MkJxr0qI96IfSeK9QL4r-Jl3ujzQnJ_UJfuXqRrW_VGZoQumTn_c4xxzNAzwEx33MP2UQeAIg',
   'hIukE6A0NxIR-RMRtgtmeNLMyRSE_9h3vI45dpCRAdkDYns6DQBvA_TIC2OvP63m

In [193]:
datetime_converter('date', 1697288568382)

datetime.datetime(2023, 10, 14, 15, 2, 48, 382000)